In [95]:
### Library Import Initialization

import numpy as np
import math
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import warnings
warnings.filterwarnings("ignore")

In [96]:
### Function to Import Stock Tickers and Calculate Final Stock Price

def import_stock_data(tickers, start_date):
    data = pd.DataFrame()
    if len([tickers]) == 1:
        data[tickers] = yf.download(tickers, start_date)['Adj Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t] = yf.download(tickers, start_date)['Adj Close']
    return data

tickers = 'GOOG'
stock_data = import_stock_data(tickers, '2018-01-01')
# Get the Current Stock Price (Starting Node of Tree)
S_0 = stock_data[tickers].iloc[-1]
print(S_0)

[*********************100%%**********************]  1 of 1 completed

153.94000244140625


In [97]:
### Sigma Calculation 

### Sigma Calculation 
def compute_sigma(data):
    # Compute the standard deviation of returns
    sigma = np.std(data) / 100
    return sigma

get_sigma = compute_sigma(stock_data)
sigma = get_sigma.values[0]
sigma

0.32998271375161553

In [98]:
### Function to Compute the Trinomial Model Option Price

def trinomial_model(S_0, K, T, r, b, sigma, option_type, n):
    """
    Parameters:
    S_0 (float): Current stock price (starting node of tree)
    K (float): Strike price of the option
    T (float): Time to expiration (in years)
    r (float): Risk-free interest rate
    sigma (float): Volatility of the underlying stock
    option_type (str): Type of option ('call' or 'put')
    n (int): Number of steps in the binomial tree
    """
    # Logic to change sign of option calculation
    if option_type == 'call':
        z = 1
    elif option_type == 'put':
        z = -1
    
    # Compute dt and Df
    dt = T / n
    # Compute up and down parameters
    u = np.exp(sigma * np.sqrt(2 * dt))
    d = np.exp(-sigma * np.sqrt(2 * dt))

    # Compute probability of Increase, Decrease, and No Change
    p_u = ((np.exp((b * dt) / 2) - np.exp(-sigma * np.sqrt(dt / 2))) / 
          (np.exp(sigma * np.sqrt(dt / 2)) - np.exp(-sigma * np.sqrt(dt / 2)))) ** 2
    p_d = ((np.exp(sigma * np.sqrt(dt / 2)) - (np.exp((b * dt) / 2))) / 
       (np.exp(sigma * np.sqrt(dt / 2)) - np.exp(-sigma * np.sqrt(dt / 2)))) ** 2
    p_m = 1 - p_u - p_d

    # Init arrays for strike prices and option values
    option_values = np.zeros(2*n + 1)

    # Calculate option values at maturity by caluclating the optimal option value at each node i for 2n+1 nodes to to reach the final time step
    for i in range(2 * n + 1):
        option_values[i] = max(0, z * (S_0 * u ** max(i - n, 0) * d ** max(n - i, 0) - K))
    
    # Backward induction to calculate option values at earlier time points
    for j in range(n - 1, -1, -1):
        for i in range(j * 2 + 1):
            option_values[i] = np.exp(-r*dt) * (p_u * option_values[i + 2] + p_m * option_values[i + 1] + p_d * option_values[i])

    return option_values[0]


In [110]:
### Function call and outputs
K = 155   # Strike price of the option
T = 0.5   # Time to expiration (in years)
r = 0.08 # Risk-free interest rate
b = 0.05 # Cost of carry
n = 100  # Number of steps in the trinomial tree
option_type = "put" 

OptionValue = trinomial_model(S_0, K, T, r, b, sigma, option_type, n)
print("The trinomial calculation gives an Option Value: " + str(round(OptionValue, 4)))


The trinomial calculation gives an Option Value: 10.1247


In [107]:
### Test Case
S_1 = 100  # Current price of the underlying asset
K = 110   # Strike price of the option
T = 0.5   # Time to expiration (in years)
r = 0.08 # Risk-free interest rate
sigma = 0.27  # Volatility
b = 0.05 # Cost of carry
n = 30  # Number of steps in the trinomial tree
option_type = "put"  # "c" for call option, "p" for put option

OptionValue = trinomial_model(S_1, K, T, r, b, sigma, option_type, n)
print("The trinomial calculation gives an Option Value: " + str(round(OptionValue, 4)) + ", and the value is supposed to be: 11.6493")

The trinomial calculation gives an Option Value: 11.8521, and the value is supposed to be: 11.6493
